# Conversational Memory

Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation. At bare minimum, a conversational system should be able to access some window of past messages directly.

We call this ability to store information about past interactions "memory". LangChain provides a lot of utilities for adding memory to a system. These utilities can be used by themselves or incorporated seamlessly into a chain.

## Chat Messages

One of the core utility classes underpinning most (if not all) memory modules is the `ChatMessageHistory` class. This is a super lightweight wrapper that provides convenience methods for saving `HumanMessages`, `AIMessages`, and then fetching them all.

In [4]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

Right now our conversation history is empty. Let's add a few messages to it.

In [5]:
history.add_user_message("I want you to act as a software engineer.")
print(history)

messages=[HumanMessage(content='I want you to act as a software engineer.')]


Think of the message we added as the first user prompt. Here's how the conversation history might look after the LLM responds. 

In [6]:
history.add_ai_message("Hello! I will act as a software engineer.")
print(history)

messages=[HumanMessage(content='I want you to act as a software engineer.'), AIMessage(content='Hello! I will act as a software engineer.')]


In reality, manipulating the conversation history manually can be a bit tedious. Luckily, LangChain provides a few utilities to make this easier.

## Conversation Buffer

This section shows how to use `ConversationBufferMemory`. This memory allows for storing messages and then extracts the messages in a variable.

In [11]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)
memory

ConversationBufferMemory(return_messages=True)

Initially, our memory is empty. Let's add a few messages to it.

In [12]:
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'), AIMessage(content='whats up')]}

## Using Memory in a Chain

Finally, let's take a look at using this in a chain (setting `verbose=True` so we can see the prompt). For this example, we will integrate the `ConversationBufferMemory` into a chain called `ConversationChain`.

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain


llm = ChatOpenAI()
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=memory,
)

conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Hi there!
AI:

> Finished chain.


'Hello! How can I assist you today?'

In [15]:
conversation.predict(input="My name is Meerkat.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi there!'), AIMessage(content='Hello! How can I assist you today?')]
Human: My name is Meerkat.
AI:

> Finished chain.


'Nice to meet you, Meerkat! How can I assist you today?'

In [16]:
conversation.predict(input="What is the largest known star in the universe?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi there!'), AIMessage(content='Hello! How can I assist you today?'), HumanMessage(content='My name is Meerkat.'), AIMessage(content='Nice to meet you, Meerkat! How can I assist you today?')]
Human: What is the largest known star in the universe?
AI:

> Finished chain.


'The largest known star in the universe is UY Scuti. It is a red supergiant located in the constellation Scutum. UY Scuti has a radius of about 1,700 times that of the Sun, making it one of the largest stars known to exist. However, please note that our understanding of the universe is constantly evolving, and there may be even larger stars yet to be discovered.'

In [17]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi there!'), AIMessage(content='Hello! How can I assist you today?'), HumanMessage(content='My name is Meerkat.'), AIMessage(content='Nice to meet you, Meerkat! How can I assist you today?'), HumanMessage(content='What is the largest known star in the universe?'), AIMessage(content='The largest known star in the universe is UY Scuti. It is a red supergiant located in the constellation Scutum. UY Scuti has a radius of about 1,700 times that of the Sun, making it one of the largest stars known to exist. However, please note that our understanding of the universe is constantly evolving, and there may be even larger stars yet to be discovered.')]
Hum

'Your name is Meerkat.'

Notice that the language model was able to remember the user name correctly. 